# Objectives:

1. Implement data preprocessing to summarize the features in the time series. The features hypothesized to be useful are as follows:
    * Band Spectrum Energy (eeg.get_power) (for EEG)
    * Fractal Dimension of Time Series (pyEEG)
    * Skewness and Kurtosis of the distributions (scipy.stats)
    * Regular stats (mean, median, max, min of time series)
    
2. Models to train:
    * Linear Chain Conditional Random Field with structured SVM solver with a 1-slack QP with L1 slack penalty (pystruct)
    * A simple SVM like Andreas (sklearn SVC)
    * A Hidden Markov model?
    
3. Cross Validate:
    * Build a proper function/system for leave-one-subject-out CV
    * Test models and ensembles on it


In [207]:
import importlib
import utils
importlib.reload(utils)
import utils
from utils import *

In [2]:
xtrain_eeg1, xtrain_eeg2, xtrain_emg, ytrain, xtest_eeg1, xtest_eeg2, xtest_emg = load_data()

Loading xtrain...
Shapes: (64800, 512) (64800, 512) (64800, 512)
Loading ytrain...
Shape: (64800, 1)
Loading xtest...
Shapes: (43200, 512) (43200, 512) (43200, 512)


In [106]:
# Test simple statistics
guy1 = xtrain_eeg1
a = simple_statistics(guy1)
print(a.shape)

(64800, 7)


In [74]:
# Test power features
guy1 = xtrain_eeg1[0:20]
guy2 = xtrain_eeg2[0:20]
wow = eeg.get_power_features(signal=guy1.T, sampling_rate=128)

# First have to join the two signals:
hah = np.vstack((guy1, guy2))
print(hah.shape)
wow2 = eeg.get_power_features(signal=hah.T, sampling_rate=128)

# plt.plot(wow[4][:, 1], alpha=0.5)
# plt.plot(wow2[4][:, 1], alpha=0.7)

for i in [1, 2, 3, 4, 5]:
    print(np.all(wow[i][:, 5] == wow2[i][:, 5]))
    
# True
# True
# True
# True
# True
# Conclusion: passing in signals separately into get_power_features is equivalent to passing them in together. 

(40, 512)
True
True
True
True
True


In [90]:
sig = guy1
[_, theta, alpha_low,alpha_high,beta, gamma]= eeg.get_power_features(signal=sig.T, sampling_rate=128)
ts1 = simple_statistics(theta.T)
al1 = simple_statistics(alpha_low.T)
ah1 = simple_statistics(alpha_high.T)
b1 = simple_statistics(beta.T)
g1 = simple_statistics(gamma.T)
print(np.concatenate((ts1, al1, ah1, b1, g1), axis=1).shape)

feats = power_features(guy1)
print(feats.shape)

(20, 35)
(20, 35)


In [97]:
# Join the first two sets of feats:
print(a.shape, feats.shape)
lol = process_EEG(guy1)
lol.shape

(20, 7) (20, 35)


(20, 42)

In [120]:
# Test andreas power features
loldamn = andreas_power_features(xtrain_eeg1)

In [125]:
lmao = process_EEG(xtrain_eeg1)

In [126]:
lmao.shape

(64800, 12)

# Model Implementation: Conditional 

In [199]:
idx = 21600*2
xtrain_eeg1_ = xtrain_eeg1[0:idx]
xtrain_eeg2_ = xtrain_eeg2[0:idx]
eeg1_ = process_EEG(xtrain_eeg1_)
eeg2_ = process_EEG(xtrain_eeg2_)
xtrain_ = np.concatenate((eeg1_, eeg2_), axis=1)
ytrain_ = ytrain[0:idx]['y']

xtrain_eeg1_ = xtrain_eeg1[idx:-1]
xtrain_eeg2_ = xtrain_eeg2[idx:-1]
eeg1_ = process_EEG(xtrain_eeg1_)
eeg2_ = process_EEG(xtrain_eeg2_)
xtest_ = np.concatenate((eeg1_, eeg2_), axis=1)
ytest_ = ytrain[idx:-1]['y']

xtest_eeg1_ = xtrain_eeg1[idx:-1]
xtest_eeg2_ = xtrain_eeg2[idx:-1]
xtest_final = 

In [204]:
# CRF

# xtrain_ = np.reshape(xtrain_, (xtrain_.shape[0], 1, xtrain_[0].shape[0])) # Reshape so that it works with CRF
xtrain_crf = np.reshape(xtrain_, (2, -1, 24)) # Reshape so that it works with CRF
ytrain_crf = np.reshape(ytrain_, (2, -1)) -1 # Reshape so that it works with CRF
# X_test_crf = X_test.reshape(2, -1, 49)
print(xtrain_crf.shape, ytrain_crf.shape)

print("Starting CRF...")
classes = np.array([1, 2, 3])
weights_crf = compute_class_weight("balanced", list(classes), ytrain_)
weights_crf[0] = weights_crf[0]+2.5
weights_crf[1] = weights_crf[1]+1.5
    
model = ChainCRF(class_weight=weights_crf)
ssvm = OneSlackSSVM(model=model, C=0.5, max_iter=2000)
    
ssvm.fit(xtrain_crf, ytrain_crf)   
   

# Test on the third guy
xtest_crf = np.reshape(xtest_, (1, -1, 24))
ytest_crf = np.reshape(ytest_, (1, -1)) -1
print(xtest_crf.shape, ytest_crf.shape)
y_pred_crf = ssvm.predict(xtest_crf)
y_pred_crf = np.asarray(y_pred_crf).reshape(-1) + 1  

print("SCORE:", sklearn.metrics.accuracy_score(ytest_, y_pred_crf))


# y_pred_filtered_crf = medfilt(y_pred_crf, FILTER_WINDOW)


# print(f"CRF unique predictions {np.unique(y_pred_filtered_crf)}")


(2, 21600, 24) (2, 21600)
Starting CRF...
(1, 21599, 24) (1, 21599)
SCORE: 0.9568035557201723


In [208]:
# Try to make a submission:


y_pred_final = ssvm.predict(xtest_crf)
make_submission("firsttrylol.csv", y_pred_crf)

ValueError: Length of values does not match length of index

In [195]:
xtrain_crf = np.reshape(xtrain_, (3, -1, 24)) # Reshape so that it works with CRF
ytrain_crf = np.reshape(ytrain_, (3, -1, 1)) # Reshape so that it works with CRF
print(xtrain_crf.shape, ytrain_crf.shape)



# xtrain_crf[0].shape
print(len(y_pred_crf))


(3, 21600, 24) (3, 21600, 1)
64800


In [ ]:
## SVC APPROACH -- GRID-SEARCH CV

steps = [("scaler", StandardScaler()), ("classifier", SVC())]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf"],
              "classifier__gamma": ["auto"],
              "classifier__C": [0.1, 0.3, 0.5, 0.7, 1, 5, 10, 50],  
              "classifier__class_weight": ["balanced"],
              "classifier__degree": [1,3,5,7]
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'balanced_accuracy', verbose = 2)

grid.fit(xtrain_eeg.values, ytrain.values.ravel())
print(grid.best_score_)
print(grid.best_params_)